In [19]:
chave_openai = "sk-z98eLFYzkYaFX5hAMzB2FrputydcOqVT9caGJsBM5mT3BlbkFJgcZ2jV7pX6kqnmw69MBKseEo9Y57E3gIPmCkz-4WUA"

In [20]:
import os
import glob
import openai
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
chave_openai = "sk-z98eLFYzkYaFX5hAMzB2FrputydcOqVT9caGJsBM5mT3BlbkFJgcZ2jV7pX6kqnmw69MBKseEo9Y57E3gIPmCkz-4WUA"
openai.api_key = chave_openai

In [22]:


# Função para extrair a seção "Dos Fatos" dos PDFs
def extrair_dos_fatos(pasta):
    # Encontrar todos os arquivos PDF na pasta especificada
    pdf_paths = glob.glob(os.path.join(pasta, '*.pdf'))

    textos_dos_fatos = []
    for pdf_path in pdf_paths:
        loader = PyPDFLoader(pdf_path)
        documentos = loader.load()
        texto_completo = "\n".join([pagina.page_content for pagina in documentos])

        # Extrair a seção "Dos Fatos"
        if 'Dos Fatos' in texto_completo:
            indice_inicio = texto_completo.find('Dos Fatos')
            indice_fim = texto_completo.find('Do Direito', indice_inicio)
            if indice_fim == -1:
                indice_fim = len(texto_completo)
            dos_fatos = texto_completo[indice_inicio:indice_fim]
            textos_dos_fatos.append(dos_fatos)
    
    return textos_dos_fatos

# Caminho da pasta com os PDFs
pasta_pdfs = r"C:\Users\Victor Broering\pycode\ia_juridico\memoria"

# Chama a função e imprime o resultado
resultados = extrair_dos_fatos(pasta_pdfs)

# Exibe os textos extraídos
for i, texto in enumerate(resultados):
    print(f"Texto extraído do arquivo {i+1}:\n{texto}\n")


In [23]:
def criar_prompt_estilo(textos_dos_fatos):
    texto_combinado = "\n".join(textos_dos_fatos)
    if len(texto_combinado) > 2000:
        texto_combinado = texto_combinado[:2000]
    prompt_estilo = f"Por favor, ao processar os textos dos PDFs, utilize o exemplo de estilo de redação fornecido a seguir como referência. Mantenha o embasamento legal e o tom jurídico apropriado. Evite o uso do termo 'menor', que pode ser considerado pejorativo; em vez disso, utilize 'infante', 'criança' ou 'adolescente', conforme o caso, se necessário.:\n{texto_combinado}\n\n"
    return prompt_estilo

In [24]:
def criar_agente(prompt_estilo):
    llm = OpenAI(model_name='gpt4-o', temperature=0.7)
    ferramentas = []
    memoria = ConversationBufferMemory(memory_key='chat_history')
    agente = initialize_agent(ferramentas, llm, agent='zero-shot-react-description', memory=memoria, verbose=True)
    prompt = PromptTemplate(
        input_variables=['input_text'],
        template=(
            f"{prompt_estilo}"
            "Agora, escreva uma resposta seguindo o mesmo estilo para os seguintes fatos fornecidos pelo usuário:\n"
            "{{input_text}}\n\n"
            "No final, indique o que precisa ser provado."
        )
    )
    llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
    return llm_chain



In [25]:
def main():
    # Caminho da pasta com os PDFs
    pasta_pdfs = r"C:\Users\Victor Broering\pycode\ia_juridico\memoria"

    # Encontrar todos os arquivos PDF na pasta especificada
    pdf_paths = glob.glob(os.path.join(pasta_pdfs, '*.pdf'))

    # Verificar se encontrou PDFs
    if not pdf_paths:
        print(f"Nenhum arquivo PDF foi encontrado na pasta {pasta_pdfs}")
        return

    # Extrair as seções "Dos Fatos"
    textos_dos_fatos = extrair_dos_fatos(pdf_paths)
    if not textos_dos_fatos:
        print("Não foi possível encontrar a seção 'Dos Fatos' nos PDFs fornecidos.")
        return

    # Criar o prompt de estilo
    prompt_estilo = criar_prompt_estilo(textos_dos_fatos)

    # Criar o agente (LLMChain)
    agente = criar_agente(prompt_estilo)

    print("Escreva os fatos da petição jurídica (ou digite 'sair' para encerrar):")
    while True:
        entrada_usuario = input()
        if entrada_usuario.lower() in ['sair', 'exit', 'quit']:
            break

        # Obter a resposta do agente
        resposta = agente.run(entrada_usuario)
        print("\nResposta Gerada:\n")
        print(resposta)

        # Perguntar se o usuário deseja salvar a resposta
        salvar = input("\nDeseja salvar a resposta em um arquivo? (s/n): ")
        if salvar.lower() == 's':
            nome_arquivo = input("Digite o nome do arquivo (sem extensão): ")
            with open(f"{nome_arquivo}.txt", 'w', encoding='utf-8') as f:
                f.write(resposta)
            print(f"Resposta salva em {nome_arquivo}.txt")


    main()